## Answering Business Questions using SQL

In this project, we're going to practice using our SQL skills to answer business questions. We'll use the Chinook database provided as a SQLite database file called `chinook.db`.

The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store. The [Chinook database](https://github.com/lerocha/chinook-database) contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. A schema diagram will help us understand the available columns and the structure of the data. Here's a schema diagram for the Chinook database:

<img src='https://s3.amazonaws.com/dq-content/191/chinook-schema.svg' style='width:600px;height:600px'/>

First, we'll connect to the database.

### Connecting to the database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Next, we'll get familiar with the data. For that, we can query the database to get a list of all tables and views in the database.

### Overview of the data

In [2]:
%%sql
SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' 
AND 
    name NOT LIKE 'sqlite_%';

Done.


name
album
artist
customer
employee
genre
invoice
invoice_line
media_type
playlist
playlist_track


It is possible and recommendable to overview some information inside each table to familiarize with its structure. For that, it is only necessary to apply the following query:

`SELECT * FROM [table] LIMIT 10;`

It is also important to indicate a limit not to overburden the search time.

### Selecting best genres to sell in the USA

Next, suppose that the Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|      Artist Name     |  Genre  |
|:--------------------:|:-------:|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [3]:
%%sql

WITH tracks_sold_usa AS 
    (
    SELECT
        il.track_id
    FROM invoice AS i
    INNER JOIN invoice_line AS il ON il.invoice_id = i.invoice_id
    WHERE i.billing_country = 'USA'
    )

SELECT
    g.name AS gender,
    COUNT(*) AS genre_purchases,
    ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM tracks_sold_usa), 3) AS percentage_purchases
FROM track AS t
INNER JOIN genre AS g ON g.genre_id = t.genre_id
INNER JOIN tracks_sold_usa AS tsu ON tsu.track_id = t.track_id
GROUP BY g.name
ORDER BY genre_purchases DESC;

Done.


gender,genre_purchases,percentage_purchases
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Latin,22,0.021
Pop,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


Looking at the results, the best selling gender is _Rock_ (53.4% of total sales) followed by _Alternative & Punk_ (12.4% of total sales) and _Metal_ (11.8% of total sales).

Taking this into account, from the previous list the three albums to add to the store should be, in order:

    - Red Tone (Punk)
    - Slim Jim Bites (Blues)
    - Meteor and the Girls (Pop)

With these 3 albums, we can cover a total of 17.9% of total sales of the market, so if we really want to have a big chance to sell an album, we should look for a _Rock_ gender one.

### Analyzing employee sales performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We can, then, analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [4]:
%%sql

WITH employee_sales AS 
    (
    SELECT 
        c.support_rep_id,
        i.total
    FROM invoice AS i
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    )
    

SELECT
    e.first_name || ' ' || e.last_name AS employee_name,
    e.hire_date,
    SUM(es.total) AS total_sales
FROM employee AS e
INNER JOIN employee_sales AS es ON es.support_rep_id = e.employee_id
GROUP BY e.employee_id
ORDER BY total_sales DESC;

Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


From the results we can extract that, while there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

### Analyzing sales by country

The next task is to analyze the sales data for customers from each different country. In particular, to calculate data, for each country, on the:

    - Total number of customers
    - Total value of sales
    - Average value of sales per customer
    - Average order value

In [5]:
%%sql

SELECT
    c.country,
    COUNT(DISTINCT c.customer_id) AS total_customers,
    ROUND(SUM(i.total), 3) AS total_sales_value,
    ROUND(SUM(i.total) / COUNT(DISTINCT c.customer_id), 3) AS avg_sales_value_per_customer,
    ROUND(SUM(i.total) / COUNT(DISTINCT i.invoice_id), 3) AS avg_order_value
FROM customer AS c
INNER JOIN invoice AS i ON i.customer_id = c.customer_id
GROUP BY country
ORDER BY total_sales_value DESC;

Done.


country,total_customers,total_sales_value,avg_sales_value_per_customer,avg_order_value
USA,13,1040.49,80.038,7.943
Canada,8,535.59,66.949,7.047
Brazil,5,427.68,85.536,7.011
France,5,389.07,77.814,7.781
Germany,4,334.62,83.655,8.161
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.52,81.84,8.769
Portugal,2,185.13,92.565,6.384
India,2,183.15,91.575,8.721
Ireland,1,114.84,114.84,8.834


Now, we are going to make a modification. All of the countries with only one customer will be gathered as "Others" in the analysis to simplify the reading.

In [6]:
%%sql

WITH customers_per_country AS
    (
    SELECT 
        country,
        COUNT(DISTINCT customer_id) AS total_customers
    FROM customer
    GROUP BY country
    ),
    countries_one_customer AS
    (
    SELECT
        country
    FROM customers_per_country
    WHERE total_customers = 1
    ),
    modified_set AS
    (
    SELECT
        c.*,
        i.*,
        CASE
            WHEN c.country IN countries_one_customer THEN 'Others'
            ELSE c.country
        END AS modified_country
    FROM customer AS c
    INNER JOIN invoice AS i ON i.customer_id = c.customer_id
    )

SELECT
    modified_country,
    COUNT(DISTINCT customer_id) AS total_customers,
    ROUND(SUM(total), 3) AS total_sales_value,
    ROUND(SUM(total) / COUNT(DISTINCT customer_id), 3) AS avg_sales_value_per_customer,
    ROUND(SUM(total) / COUNT(DISTINCT invoice_id), 3) AS avg_order_value
FROM 
    (
    SELECT
        modified_set.*,
        CASE
            WHEN modified_country = 'Others' THEN 1
            ELSE 0
        END AS sort
    FROM modified_set
    )
GROUP BY modified_country
ORDER BY sort, total_sales_value DESC;

Done.


modified_country,total_customers,total_sales_value,avg_sales_value_per_customer,avg_order_value
USA,13,1040.49,80.038,7.943
Canada,8,535.59,66.949,7.047
Brazil,5,427.68,85.536,7.011
France,5,389.07,77.814,7.781
Germany,4,334.62,83.655,8.161
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.52,81.84,8.769
Portugal,2,185.13,92.565,6.384
India,2,183.15,91.575,8.721
Others,15,1094.94,72.996,7.449


Based on the results of the data, there may be opportunity in the following countries:

    - Czech Republic
    - United Kingdom
    - India

This is it because these countries have the highest average order value among all of them. In case of market development, it should be adequate to explore these markets first.

It's worth keeping in mind that the amount of data from each of these countries is relatively low, so we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.


### Albums vs Individual tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

    - Purchase a whole album
    - Purchase a collection of one or more individual tracks

We are going to find out what percentage of purchases are individual tracks vs whole albums. In order to answer the question, we are going to have to identify whether each invoice has all the tracks from an album.

In [25]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id,
         MIN(il.track_id) AS first_track_id
     FROM invoice_line AS il
     GROUP BY invoice_id
    )

SELECT
    album_purchase,
    COUNT(DISTINCT invoice_id) AS number_of_invoices,
    CAST(count(DISTINCT invoice_id) AS FLOAT) / (
                                        SELECT COUNT(*) FROM invoice
                                        ) AS percent_of_total_invoices
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT 
                     t.track_id 
                  FROM track AS t
                  WHERE t.album_id = (
                                      SELECT 
                                          t2.album_id 
                                      FROM track AS t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                  EXCEPT 
                  SELECT 
                     il2.track_id
                  FROM invoice_line AS il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT 
                     il2.track_id 
                  FROM invoice_line AS il2
                  WHERE il2.invoice_id = ifs.invoice_id
                  EXCEPT 
                  SELECT 
                     t.track_id 
                  FROM track AS t
                  WHERE t.album_id = (
                                      SELECT 
                                          t2.album_id 
                                      FROM track AS t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN 'yes'
             ELSE 'no'
         END AS album_purchase
     FROM invoice_first_track AS ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent_of_total_invoices
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, we could develop some different strategies: 

   - Purchasing only popular songs from the albums instead of complete albums, as they represent the 81% of total purchases. To recommend this, it would be necessary to analyze if the individual songs purchased correspond to the most popular songs from each album.
   
   - Against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.
   
Again, it would be possible to extend the analyze by answering some other questions like:

- Which artist is used in the most playlists?
- How many tracks have been purchased vs not purchased?
- Is the range of tracks in the store reflective of their sales popularity?
- Do protected vs non-protected media types have an effect on popularity?